In [ ]:
!python --version

In [ ]:
import ultralytics
ultralytics.__version__

In [ ]:
import torch
torch.__version__

In [ ]:
%cd deep_sort

In [ ]:
from ultralytics import YOLO

import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np


In [ ]:
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)




In [ ]:
!pip install easydict
from utils.parser import get_config
from deep_sort import DeepSort
from sort.tracker import Tracker

deep_sort_weights = 'deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

In [ ]:
video_path = "C:/Users/Resh/Downloads/peoplecount1/peoplecount1.mp4"

cap = cv2.VideoCapture(video_path)

In [ ]:
import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np
import pandas as pd

In [ ]:
#model=YOLO('yolov8s.pt')

In [ ]:
area1=[(312,388),(289,390),(474,469),(497,462)]

area2=[(279,392),(250,397),(423,477),(454,469)]
#area1=[(312,388),(497,390),(474,469),(497,462)]


#area2=[(279,392),(454,450),(423,477),(454,469)]
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

In [ ]:
cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap=cv2.VideoCapture("C:/Users/Resh/Downloads/peoplecount1/peoplecount1.mp4")
if not cap.isOpened():
    print("Error: Could not open video source.")
    

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (1020, 500))


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
frames = []

unique_track_ids = set()

In [ ]:
i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()

In [ ]:
my_file = open("coco.txt")
data = my_file.read()
class_list = data.split("\n") 


In [ ]:
people_entering={}
entering=set()
people_leaving={}
leaving=set()
frames = []
count=0

red_color = (0, 0, 255)  # (B, G, R)
blue_color = (255, 0, 0)  # (B, G, R)
green_color = (0, 255, 0)  # (B, G, R)


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        
        og_frame = cv2.resize(frame,(1020,500))
        frame = og_frame.copy()

        model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

        results = model(frame, device='cpu', classes=0, conf=0.8)
        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        for result in results:
            boxes = result.boxes  # Boxes object for bbox outputs
            probs = result.probs  # Class probabilities for classification outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            for class_index in cls:
                class_name = class_names[int(class_index)]
                #print("Class:", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)
        
        tracks = tracker.update(bboxes_xywh, conf, og_frame)
        
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
            w = x2 - x1  # Calculate width
            h = y2 - y1  # Calculate height
            
            results1=cv2.pointPolygonTest(np.array(area2,np.int32),((x2,y2)),False)
            if results1>=0:
                people_entering[track_id]=(x2,y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            if track_id in people_entering:
                results2=cv2.pointPolygonTest(np.array(area1,np.int32),((x2,y2)),False)
                if results2>=0:
                    cv2.rectangle(og_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.circle(og_frame,(x2,y2),5,(255,0,255),-1)
                    cv2.putText(og_frame, str(track_id), (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    entering.add(track_id)
                    
            #people Leaving
            results3=cv2.pointPolygonTest(np.array(area1,np.int32),((x2,y2)),False)
            if results3>=0:
                people_leaving[track_id]=(x2,y2)
                cv2.rectangle(og_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            if track_id in people_leaving:
                results4=cv2.pointPolygonTest(np.array(area2,np.int32),((x2,y2)),False)
                if results4>=0:
                    cv2.rectangle(og_frame, (x1, y1), (x2, y2), (255,0 , 255), 2)
                    cv2.circle(og_frame,(x2,y2),5,(255,0,255),-1)
                    cv2.putText(og_frame, str(track_id), (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    leaving.add(track_id)
                    
                    
            

            # Set color values for red, blue, and green
            red_color = (0, 0, 255)  # (B, G, R)
            blue_color = (255, 0, 0)  # (B, G, R)
            green_color = (0, 255, 0)  # (B, G, R)

            # Determine color based on track_id
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color

            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

            text_color = (0, 0, 0)  # Black color for text
            cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            # Add the track_id to the set of unique track IDs
            unique_track_ids.add(track_id)

        # Update the person count based on the number of unique track IDs
        person_count = len(unique_track_ids)

        # Update FPS and place on frame
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time
            
        cv2.polylines(og_frame,[np.array(area1,np.int32)],True,(255,0,0),2)
        cv2.putText(og_frame,str('1'),(504,471),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)

        cv2.polylines(og_frame,[np.array(area2,np.int32)],True,(255,0,0),2)
        cv2.putText(og_frame,str('2'),(466,485),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)
    
        cv2.putText(og_frame,str(len(entering)-len(leaving)),(60,80),cv2.FONT_HERSHEY_COMPLEX,(0.7),(0,0,255),2)

        # Draw person count on frame
        cv2.putText(og_frame, f"Person Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Append the frame to the list
        frames.append(og_frame)

        # Write the frame to the output video file
        out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))

        # Show the frame
        #cv2.imshow("Video", og_frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
        out.write(og_frame)
        cv2.imshow("RGB", og_frame)
        if cv2.waitKey(1)&0xFF==27:        
            break

cap.release()
out.release()
cv2.destroyAllWindows()

In [17]:
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        
        og_frame = cv2.resize(frame,(1020,500))
        frame = og_frame.copy()

        model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

        results=model.predict(frame)
        cls=None
        conf=None
        xyxy=None
        xywh=None
        
        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        people_results = [result for result in results if class_names[int(result.boxes.cls[torch.argmax(result.boxes.conf).item()])] == 'person']
        
        
        for i in people_results:
            boxes = i.boxes  # Boxes object for bbox outputs
            probs = i.probs  # Class probabilities for classification outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            for class_index in cls:
                class_name = class_names[int(class_index)]
                #print("Class:", class_name) boxes = result.boxes  # Boxes information
    
      # Extract bounding boxes for people detections
            bboxes_xywh = boxes.xywh.cpu().numpy()  # Assuming xywh format

      # Update tracker with person detections (replace your tracker if needed)
            tracks = tracker.update(bboxes_xywh,conf, og_frame)  # Update tracker (modify if necessary)

    
        
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
            w = x2 - x1  # Calculate width
            h = y2 - y1  # Calculate height
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color
            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)
            text_color = (0, 0, 0)  # Black color for text
            cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            # Add the track_id to the set of unique track IDs
            unique_track_ids.add(track_id)
            
            results1=cv2.pointPolygonTest(np.array(area2,np.int32),((x2,y2)),False)
            if results1>=0:
                people_entering[track_id]=(x2,y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            if track_id in people_entering:
                results2=cv2.pointPolygonTest(np.array(area1,np.int32),((x2,y2)),False)
                if results2>=0:
                    cv2.rectangle(og_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.circle(og_frame,(x2,y2),5,(255,0,255),-1)
                    cv2.putText(og_frame, str(track_id), (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    entering.add(track_id)
                    
            #people Leaving
            results3=cv2.pointPolygonTest(np.array(area1,np.int32),((x2,y2)),False)
            if results3>=0:
                people_leaving[track_id]=(x2,y2)
                cv2.rectangle(og_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            if track_id in people_leaving:
                results4=cv2.pointPolygonTest(np.array(area2,np.int32),((x2,y2)),False)
                if results4>=0:
                    cv2.rectangle(og_frame, (x1, y1), (x2, y2), (255,0 , 255), 2)
                    cv2.circle(og_frame,(x2,y2),5,(255,0,255),-1)
                    cv2.putText(og_frame, str(track_id), (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    leaving.add(track_id)
                    
                    
            

            # Set color values for red, blue, and green
                
            # Determine color based on track_id
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color

                #cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

                #text_color = (0, 0, 0)  # Black color for text
                #cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            # Add the track_id to the set of unique track IDs
                #unique_track_ids.add(track_id)

        # Update the person count based on the number of unique track IDs
        person_count = len(unique_track_ids)

        # Update FPS and place on frame
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time
            
        cv2.polylines(og_frame,[np.array(area1,np.int32)],True,(255,0,0),2)
        cv2.putText(og_frame,str('1'),(504,471),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)

        cv2.polylines(og_frame,[np.array(area2,np.int32)],True,(255,0,0),2)
        cv2.putText(og_frame,str('2'),(466,485),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)
    
        cv2.putText(og_frame,str(len(entering)-len(leaving)),(60,80),cv2.FONT_HERSHEY_COMPLEX,(0.7),(0,0,255),2)

        # Draw person count on frame
        cv2.putText(og_frame, f"Person Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Append the frame to the list
        frames.append(og_frame)

        # Write the frame to the output video file
        out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))

        # Show the frame
        #cv2.imshow("Video", og_frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
        out.write(og_frame)
        cv2.imshow("RGB", og_frame)
        if cv2.waitKey(1)&0xFF==27:        
            break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 320x640 1 car, 1053.1ms
Speed: 7.0ms preprocess, 1053.1ms inference, 5.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 678.0ms
Speed: 7.0ms preprocess, 678.0ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 809.1ms
Speed: 5.0ms preprocess, 809.1ms inference, 5.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 763.1ms
Speed: 5.0ms preprocess, 763.1ms inference, 4.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 car, 701.1ms
Speed: 6.0ms preprocess, 701.1ms inference, 6.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 car, 1051.1ms
Speed: 5.0ms preprocess, 1051.1ms inference, 5.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 car, 1153.1ms
Speed: 29.0ms preprocess, 1153.1ms inference, 12.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 car, 1180.1ms
Speed: 7.0ms preprocess, 1180.1ms inference, 5.0

[796, 353]


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        
        og_frame = cv2.resize(frame,(1020,500))
        frame = og_frame.copy()

        model = YOLO("yolov8s.pt")  # load a pretrained model (recommended for training)

        results=model.predict(frame)
        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        for result in results:
            boxes = result.boxes  # Boxes object for bbox outputs
            probs = result.probs  # Class probabilities for classification outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            for class_index in cls:
                class_name = class_names[int(class_index)]
                #print("Class:", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)
        
        tracks = tracker.update(bboxes_xywh, conf, og_frame)
        
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
            w = x2 - x1  # Calculate width
            h = y2 - y1  # Calculate height
            
            results1=cv2.pointPolygonTest(np.array(area2,np.int32),((x2,y2)),False)
            if results1>=0:
                people_entering[track_id]=(x2,y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            if track_id in people_entering:
                results2=cv2.pointPolygonTest(np.array(area1,np.int32),((x2,y2)),False)
                if results2>=0:
                    cv2.rectangle(og_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.circle(og_frame,(x2,y2),5,(255,0,255),-1)
                    cv2.putText(og_frame, str(track_id), (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    entering.add(track_id)
                    
            #people Leaving
            results3=cv2.pointPolygonTest(np.array(area1,np.int32),((x2,y2)),False)
            if results3>=0:
                people_leaving[track_id]=(x2,y2)
                cv2.rectangle(og_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            if track_id in people_leaving:
                results4=cv2.pointPolygonTest(np.array(area2,np.int32),((x2,y2)),False)
                if results4>=0:
                    cv2.rectangle(og_frame, (x1, y1), (x2, y2), (255,0 , 255), 2)
                    cv2.circle(og_frame,(x2,y2),5,(255,0,255),-1)
                    cv2.putText(og_frame, str(track_id), (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    leaving.add(track_id)
                    
                    
            

            # Set color values for red, blue, and green
            red_color = (0, 0, 255)  # (B, G, R)
            blue_color = (255, 0, 0)  # (B, G, R)
            green_color = (0, 255, 0)  # (B, G, R)

            # Determine color based on track_id
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color

            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

            text_color = (0, 0, 0)  # Black color for text
            cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            # Add the track_id to the set of unique track IDs
            unique_track_ids.add(track_id)

        # Update the person count based on the number of unique track IDs
        person_count = len(unique_track_ids)

        # Update FPS and place on frame
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time
            
        cv2.polylines(og_frame,[np.array(area1,np.int32)],True,(255,0,0),2)
        cv2.putText(og_frame,str('1'),(504,471),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)

        cv2.polylines(og_frame,[np.array(area2,np.int32)],True,(255,0,0),2)
        cv2.putText(og_frame,str('2'),(466,485),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)
    
        cv2.putText(og_frame,str(len(entering)-len(leaving)),(60,80),cv2.FONT_HERSHEY_COMPLEX,(0.7),(0,0,255),2)

        # Draw person count on frame
        cv2.putText(og_frame, f"Person Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Append the frame to the list
        frames.append(og_frame)

        # Write the frame to the output video file
        out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))

        # Show the frame
        #cv2.imshow("Video", og_frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
        out.write(og_frame)
        cv2.imshow("RGB", og_frame)
        if cv2.waitKey(1)&0xFF==27:        
            break

cap.release()
out.release()
cv2.destroyAllWindows()